In [1]:
#importing libraries
from bs4 import BeautifulSoup as bs 
import pandas as pd   
import numpy as np
import os

In [2]:
#Get files names to be able to read each file 
files_names = os.listdir(r'C:\\Users\\hp\\Downloads\\ML_Data\\data')

In [5]:
#Initiate bs_dict the dictionary that will cotain the data
bs_dict = dict()

In [6]:
#Crate a dictionary with the folder number as the key and the bs object as the value
for file_name in files_names: 
    #open file according to file_name
    file = open( ('C:\\Users\\hp\\Downloads\\ML_Data\\data\\' + file_name), 'r', encoding = 'utf-8') 
    #set the reference point in the beginning of the file    
    file.seek(0)      
    #store file content 
    txt = file.read()
    #conver txt into a Beautiful Soup object
    bs_obj = bs(txt) 
    #store the object in bs_dict dictionary
    bs_dict[file_name] = bs_obj

### Delete files doesn't contain usefull data 

In [47]:
#define bs_dict_keys a list of files names 
bs_dict_keys = list(bs_dict.keys())
#for each file check if it contains class notice then drop it from bs_dict, otherwise continue
for file in bs_dict_keys:
    if(bs_dict[file].find(class_ = "notice")):
        bs_dict.pop(file)
    else: 
        continue 

In [48]:
def get_value(file_number, feature, class_name):   
    """
    a function to extract a particular values from a particular file, return nan if it doesnot exist
    inputs:
        file_number(str): file name want to search in it's content
        feature(str): feature name to return it's value
        class_name(str): name of the class the feature exists in
    returns:    
        value of the given feature according to file_name, class_name.
        nan if value of the given feature doesnot exist in the file.
    """
    #define additions a list of 8 car additions names
    additions = [ 'مُكيّف', 'إغلاق مركزي', 'جهاز إنذار', 'مسجل CD', 'فتحة سقف', 'جنطات مغنيسيوم', 'فرش جلد', 'وسادة حماية هوائية' ]
    #define value with nan as a default value  
    value = np.NaN
    #check if the class_name is driving-table then get value of الموديل or موديل سنة 
    if class_name == "driving-table":   
        if feature == 'الموديل':   
            #get the h3 tag text which contains the value of الموديل
            value = bs_dict[file_number].find(class_ = class_name).find("h3").text
        elif feature == 'موديل سنة':
            #store the result of h5 tags in h5_list 
            h5_list = bs_dict[file_number].find(class_ = class_name).findAll("h5")
            #get the tag contains 'موديل سنة' to get the year
            for item in h5_list:  
                if 'موديل سنة' in item.text:
                    #split item(the tag) text according to 'موديل سنة' which will return an array of the year and a whitespace in the first index 
                    value = int(item.text.split("موديل سنة")[1])
    #check if the class_name is post-price to get the car price 
    elif class_name == "post-price":
        value = bs_dict[file_number]\
                        .find(class_ = class_name)\
                        .text.split("\n")[0]\
                        .split(' ')[0] 
    #else for the list_ads class 
    else:
        #check if the feature is سعر التأمين then get the second tag of class list_ads 
        if feature == "سعر التأمين":
            #get the value of سعر التأمين which exists in the 19th index after splitting the text based on '\n'  
            value = int(bs_dict[file_number].findAll(class_ = "list_ads")[1].text.split('\n')[19])
        else:
            #define features_list, a list contains features values existed in the first element
            #that has class 'list_ads' for the car 
            features_list = bs_dict[file_number].find(class_ = class_name).text.split("\n")
            #check if the feature is in additions list
            if feature in additions:
                #check if the feature is in features_list list so the value will 1
                if feature in features_list:
                    value = 1 
                else:        
                    value = 0
            #else for features that isn't in additions 
            else:
                #
                for match in features_list: 
                    if feature in match:  
                        value = match.split(feature)[1] 
    return value 

In [49]:
#define cars_info a list of dictionaries each one represents a particular car 
cars_info = []
#get features value for each car stored in bs_dict
for current_file_number in list(bs_dict.keys()): 
    cuurent_car_info = {  
        "السعر": float(get_value(current_file_number,"السعر", "post-price")),
        "الموديل": get_value(current_file_number , "الموديل", "driving-table"),
        'موديل سنة': get_value(current_file_number , 'موديل سنة', "driving-table"),
        "لون السيارة": get_value(current_file_number,"لون السيارة", "list_ads"),
        'قوة الماتور': float(get_value(current_file_number,'قوة الماتور', "list_ads")),
        "عدد الركاب": get_value(current_file_number, "عدد الركاب", "list_ads"),  
        "عداد السيارة": get_value(current_file_number,"عداد السيارة", "list_ads"),
        "أصحاب سابقون": get_value(current_file_number, "أصحاب سابقون", "list_ads"),
        'أصل السيارة': get_value(current_file_number,'أصل السيارة', "list_ads"),
        'رخصة السيارة': get_value(current_file_number,'رخصة السيارة', "list_ads"),
        "نوع الوقود": get_value(current_file_number,"نوع الوقود", "list_ads"),
        'نوع الجير': get_value(current_file_number,'نوع الجير', "list_ads"),  
        'الزجاج': get_value(current_file_number,'الزجاج', "list_ads"), 
        "سعر التأمين": get_value(current_file_number, "سعر التأمين", "list_ads"),
        'وسادة حماية هوائية': get_value(current_file_number, 'وسادة حماية هوائية', "list_ads"),
        'فرش جلد': get_value(current_file_number, 'فرش جلد', "list_ads"),   
        'جنطات مغنيسيوم': get_value(current_file_number,'جنطات مغنيسيوم', "list_ads"),
        'فتحة سقف': get_value(current_file_number,'فتحة سقف', "list_ads"),
        'مسجل CD': get_value(current_file_number,'مسجل CD', "list_ads"),    
        'إغلاق مركزي': get_value(current_file_number,'إغلاق مركزي', "list_ads"),
        'مُكيّف': get_value(current_file_number,'مُكيّف', "list_ads"), 
        'جهاز إنذار': get_value(current_file_number,'جهاز إنذار', "list_ads"),
    }   
    cars_info.append(cuurent_car_info)      

In [50]:
#define data, a data frame contains cars data  
data = pd.DataFrame(cars_info)

In [ ]:
#Write data to cars_data file
data.to_csv('../data/cars_data.csv', index = False)  